# Análise dos imoveis Balneário Camboriú
> Data: 16/01/2022

In [67]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [68]:
html = urlopen('https://www.tresas.com.br/todos-os-imoveis')
cf = bs(html.read(), 'html.parser')
dados_imoveis = cf.findAll('a', {'class': 'listing-card-2 max-400 w-inline-block'})
dados_imoveis[0]

<a class="listing-card-2 max-400 w-inline-block" href="/imoveis/apartamento-vista-para-o-mar-no-bairro-pioneiros-bc"><div class="listing-2-card-top"><img alt="" class="listing-card-2-photo" height="400" loading="lazy" sizes="(max-width: 479px) 88vw, (max-width: 991px) 380px, (max-width: 1439px) 28vw, 400px" src="https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62c08d480babd5d5c5bd6b72_7.jpg" srcset="https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62c08d480babd5d5c5bd6b72_7-p-500.jpeg 500w, https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62c08d480babd5d5c5bd6b72_7-p-800.jpeg 800w, https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62c08d480babd5d5c5bd6b72_7-p-1080.jpeg 1080w, https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62c08d480babd5d5c5bd6b72_7.jpg 1280w" width="400"/><div class="listing-2-price"><div class="listing-2-price-text">R$ 7.400.000</div></div></div><div class="listing-content-2"><div><div class="text-medium cod-imovel">#AP0242-BEM

In [69]:
imoveis = []
for dados in dados_imoveis:
    novos_dados = []
    novos_dados.append(dados.find('div', {'class': 'text-large'})) #descrição
    novos_dados.append(dados.find('div', {'class': 'listing-2-price-text'})) #preço
    novos_dados.append(dados.find('div', {'class': 'text-medium margin-top-4'})) #localização
    novos_dados.append(dados.find('div', {'class': 'listing-2-card-features'})) #caracteristicas do imóvel
    novos_dados.append(dados.img['src']) #link imagem
    imoveis.append(novos_dados)

In [70]:
imoveis_df = pd.DataFrame(imoveis) 
imoveis_df

,0,1,2,3,4
0,[Apartamento com Vista Mar no Bairro Pioneiros...,[R$ 7.400.000],"[Pioneiros, Balneário Camboriú - SC]","[[[3 Suítes]], [[4 Banheiros]], [[]]]",https://uploads-ssl.webflow.com/629007f9f06b5e...
1,"[Apartamento no Diamond Crystal Tower, Centro ...","[R$ 2.989.188,00]","[Rua 1201, 432. Centro - Balneário Camboriú]","[[[4 Suítes]], [[]], [[173,79m²]]]",https://uploads-ssl.webflow.com/629007f9f06b5e...
2,"[Terreno à venda em São Judas, Itajaí ]","[R$ 1.830.000,00]","[Sao Judas, Itajaí - SC]","[[[]], [[]], [[]]]",https://uploads-ssl.webflow.com/629007f9f06b5e...
3,[Apartamento com vista panorâmica e 3 suítes n...,"[R$ 6.341.183,87]","[Praia Brava, Itajaí - SC]","[[[3 Suítes]], [[3 Banheiros]], [[168m² Área P...",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"[Apartamento com 3 dormitórios sendo 2 suítes,...",[R$ 3.400.000],"[Centro, Balneário Camboriú - SC]","[[[1 Suíte]], [[2 Banheiros]], [[110m² Área Pr...",https://uploads-ssl.webflow.com/629007f9f06b5e...
...,...,...,...,...,...
64,"[Apartamento no Edifício Le Visage Divinité, C...",[R$ 5.200.000],"[Centro, Balneário Camboriú - SC]","[[[4 Suítes]], [[5 Banheiros]], [[170m² Área P...",https://uploads-ssl.webflow.com/629007f9f06b5e...
65,[Apartamento com 3 suítes modernas no Centro d...,"[R$ 2.376.347,09]","[Centro, Balneário Camboriú - SC]","[[[3 Suítes]], [[3 Banheiros]], [[]]]",https://uploads-ssl.webflow.com/629007f9f06b5e...
66,[Apartamento com 3 Suítes no Elegance Tower em...,"[R$ 3.650.000,00]","[Rua 3618,29, Centro - Balneário Camboriú/SC]","[[[3 Suítes]], [[]], [[137,17 m² de Área Priva...",https://uploads-ssl.webflow.com/629007f9f06b5e...
67,[Apartamento com 4 suítes no Centro de Balneá...,[R$ 4.700.000],"[Centro, Balneário Camboriú - SC]","[[[4 Suítes]], [[5 Banheiros]], [[177m² Área P...",https://uploads-ssl.webflow.com/629007f9f06b5e...
